In [1]:
import numpy as np
import matplotlib.pyplot as plt
from emitter import Particle, Emitter

In [28]:
cmin = 0
cmax = 2
n_points = 20

In [29]:
c = np.linspace(cmin, cmax, n_points)

In [30]:
grid = np.array([(c[i], c[j]) for j in range(n_points) for i in range(n_points)])

In [31]:
y = lambda x: .2 * x ** 3 - .8 * x + .7
fx = y(c)

In [32]:
class Cube:
    def __init__(self, coords, state="empty"):
        """ Cube is defined by a down-bottom-left point """
        self.c = coords.copy()
        self.state = state

    def __repr__(self):
        return f"[{self.coords.round(2)}] - {self.state}"

    @property
    def coords(self):
        return self.c

    @coords.setter
    def coords(self, val):
        self.c = val

    def is_empty(self):
        return self.state == 'empty'
    
    def flush(self):
        if self.state == 'water':
            self.state = 'empty'

In [33]:
class HeightMap:
    def __init__(self, cmin, cmax, n_points, n_dims=2, estimator=None):
        self.n = n_points
        self.n_dims = n_dims
        self.hmap = np.empty((self.n ** n_dims,), dtype=object)
        
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = c[1] - c[0]
        
        for i in range(self.n ** n_dims):
            idx = divmod(i, n_points)
            cp = np.array([c[_] for _ in idx])
            self.hmap[i] = Cube(cp, 'terrain' if self.under_f(y, cp) else 'empty')
    
    def dn(self, coords):
        """ n-dimentional index convertion to 1D array """
        n = len(coords)
        if np.any([_ >= self.n or _ < 0 for _ in coords]):
            with out:
                raise ValueError("DN index out of bounds:", coords)
        return self.hmap[sum([_ * self.n ** p for _, p in zip(coords, range(n - 1, -1, -1))])]
    
    def find_obstacle(self, x):
        """ Find top-level obstacle
         Coords should specify subspace without the last coordinate 
         """
        # out-of-map test
        for i in range(self.n - 1, -1, -1):
            if self[x, i].state != 'empty':
                return self[x, i]
        print("Nil hit while looking for an obstacle")
        return self[idx]

    def d1_to_dn(self, i):
        """ D1 index to DN index conversion """
        if i >= self.hmap.shape[0]:
            raise ValueError(f"D1 index out of bounds: {i} >= {self.hmap.shape[0]}")
        coords = np.zeros((self.n_dims), dtype=np.int32)
        for _ in range(self.n_dims - 1, -1, -1):
            coords[_] = int(i % self.n)
            i /= self.n
        print("Coords:", coords)
        return self[coords]
        
    def under_f(self, f, point):
        f_val = f(point[:-1])
        return point[-1] <= f_val
    
    def reset(self):
        for cube in self.hmap:
            if cube.state == 'water':
                cube.state = 'empty'
    
    def __getitem__(self, item):
        if type(item) is int:
            return self.hmap[item]
        elif hasattr(item, '__len__'):
            if len(item) == self.n_dims:
                return self.dn(item)
            else:
                print("Dims must match!")
                raise ValueError(f"Dimensions must match when indexing obj[{item}]")

In [34]:
class Simulation:
    def __init__(self, hmap, cmin=0, cmax=2, n_points=20):
        self.particles = []
        self.dt = 0.003
        self.cmin = cmin
        self.cmax = cmax
        self.n_grid = n_points
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = self.c[1] - self.c[0]
        self.hmap = hmap
        self.gradient = True
        self.collision_loss = .8  # loss in velocity when collision occurs
    
    def add_particles(self, particles):
        self.particles.extend(particles)
    
    def add_particle(self, particle):
        self.particles.append(particle)
    
    def correct_location(self, xcur, ycur, xnext, ynext):
        with out:
            print(f"Correcting location: cur [{xcur}, {ycur}] next [{xnext}, {ynext}]")
            ydf = ynext - ycur
            xdf = xnext - xcur
            ystep = -1 if ycur > ynext else 1
            xstep = -1 if xcur > xnext else 1
            while ycur != ynext or xcur != xnext:
                print(f"Candidate location: [{xcur}, {ycur}]")
                if not self.hmap[(xcur + xstep, ycur)].is_empty():
                    print(f"Corrected location: [{xcur - xstep}, {ycur}]: {self.hmap[xcur - xstep, ycur]}")
                    return xcur - xstep, ycur
                elif not self.hmap[(xcur, ycur + ystep)].is_empty():
                    print(f"Corrected location: [{xcur}, {ycur - ystep}]: {self.hmap[xcur, ycur - ystep]}")
                    return xcur, ycur - ystep
                elif not self.hmap[(xcur + xstep, ycur + ystep)].is_empty():
                    print(f"Corrected location: [{xcur - xstep}, {ycur - ystep}]: {self.hmap[xcur - xstep, ycur - ystep]}")
                    return xcur - xstep, ycur - ystep
                ycur += ystep
                xcur += xstep        
                
    
    def update_particles(self):
        # 1) gravity
#         with out:
            for particle in self.particles:
                cur_loc = self.discritize(particle)
                self.hmap[cur_loc].flush()
#               print("Trying destination loc:", particle + particle.grad)
                next_loc = self.discritize(particle + particle.grad)

                if cur_loc != next_loc and self.hmap.dn(next_loc).state != 'empty':
                    ground_level = next_loc
#                 next_loc = self.correct_location(*cur_loc, *next_loc)
                # ground is somewhere between cur_loc and next_loc
                    for i in range(cur_loc[1] - 1, next_loc[1] - 1, -1):
                        print(f"Loc: {cur_loc[0], i} - {hmap.dn((cur_loc[0], i)).state}")
                        if hmap.dn((cur_loc[0], i)).state != 'empty':
                            next_loc = (cur_loc[0], i + 1)
                            break
                    if not self.hmap[next_loc].is_empty():
                        next_loc = cur_loc
#                   print("Corrected next loc:", next_loc, self.hmap[next_loc])
                    particle.grad -= particle.grad * self.collision_loss
                    if self.gradient:  # calculate_gradient
                        left_obstacle = self.hmap.find_obstacle(next_loc[0] - 1) if next_loc[0] > 0 else None
                        right_obstacle = self.hmap.find_obstacle(next_loc[0] + 1) if next_loc[0] < (self.n_grid - 1) else None
                        dydx = particle.y - right_obstacle.coords[1] if right_obstacle is not None else 0
                        dydx = (dydx + (particle.y - left_obstacle.coords[1])) / 2 if left_obstacle is not None else dydx
#                       print(f"Left: {left_obstacle} | Right: {right_obstacle} | Cur_pos: {cur_loc} | dydx: {dydx}")
                        particle.grad[0] -= dydx * .1
#                         print(f"after: {particle}")
                    nloc = self.hmap.dn(next_loc)
                    particle.coords = nloc.coords + self.step / 2  # move in the middle of the first empty cube
                else:
#                   print(f"No collision, particle: {particle}")
                    particle.grad[((particle.coords + particle.grad) < self.cmin) | ((particle.coords + particle.grad) > self.cmax)] = 0
                    particle.coords += particle.grad
                    particle.grad[-1] += -9.81 * self.dt
                if self.hmap[self.discritize(particle)].is_empty():
                    self.hmap[self.discritize(particle)].state = 'water'
                else:
                    print("Something went bad! State of current particle is:", self.hmap[self.discritize(particle)])
    
    def discritize(self, particle):
        v = []
        for c in particle.coords:
            if c < self.cmin:
                c = self.cmin
            elif c >= self.cmax:
                c = self.cmax - .01
            v.append(int(c // self.step))
        return v
    
    def reset(self):
        self.particles = []

In [35]:
hmap = HeightMap(cmin, cmax, n_points)

coords = np.array([hmap.hmap[i].coords for i in range(hmap.n ** 2)])
col = ['green' if hmap.hmap[i].state == 'terrain' else 'steelblue' for i in range(hmap.n ** 2)]


e = Emitter(cmin, cmax, ppersec=60, n_coords=2)
# e.set_origin(np.array((1.8, 2)))

s = Simulation(hmap, cmin=cmin, cmax=cmax, n_points=n_points)


In [36]:
from D1_viz import D1_viz
d = D1_viz(s, e)
out = d.out
d.run()